Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

Tune L2 regularization for logistic model

In [5]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # L2 regularization for the fully connected parameters.
  regularizers = tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases) 
  # Add the regularization term to the loss.
  loss += 5e-4 * regularizers
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 20.073441
Minibatch accuracy: 11.7%
Validation accuracy: 12.5%
Minibatch loss at step 500: 2.079197
Minibatch accuracy: 77.3%
Validation accuracy: 75.8%
Minibatch loss at step 1000: 1.744632
Minibatch accuracy: 80.5%
Validation accuracy: 77.4%
Minibatch loss at step 1500: 1.098860
Minibatch accuracy: 84.4%
Validation accuracy: 78.4%
Minibatch loss at step 2000: 1.023061
Minibatch accuracy: 80.5%
Validation accuracy: 79.2%
Minibatch loss at step 2500: 1.092160
Minibatch accuracy: 76.6%
Validation accuracy: 80.4%
Minibatch loss at step 3000: 0.897657
Minibatch accuracy: 82.0%
Validation accuracy: 80.9%
Test accuracy: 88.4%


Tune L2 regularization for neural network model

In [7]:
batch_size = 128
hidden_layer_size = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weightsHidden = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer_size]))
  biasesHidden = tf.Variable(tf.zeros([hidden_layer_size]))
    
  weights = tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  logitsHidden = tf.matmul(tf_train_dataset, weightsHidden) + biasesHidden
  hiddenLayer = tf.nn.relu(logitsHidden)

  # Training computation.
  logits = tf.matmul(hiddenLayer, weights) + biases
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # L2 regularization for the fully connected parameters.
  regularizers = (tf.nn.l2_loss(weightsHidden) + tf.nn.l2_loss(biasesHidden) +
                  tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases))
  # Add the regularization term to the loss.
  loss += 5e-4 * regularizers
 
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
    
  valid_hidden = tf.nn.relu(tf.matmul(tf_valid_dataset, weightsHidden) + biasesHidden)
  valid_logits = tf.matmul(valid_hidden, weights) + biases
  valid_prediction = tf.nn.softmax(valid_logits)
    
  test_hidden = tf.nn.relu(tf.matmul(tf_test_dataset, weightsHidden) + biasesHidden)
  test_logits = tf.matmul(test_hidden, weights) + biases
  test_prediction = tf.nn.softmax(test_logits)

In [8]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 471.523987
Minibatch accuracy: 14.8%
Validation accuracy: 21.1%
Minibatch loss at step 500: 134.516937
Minibatch accuracy: 81.2%
Validation accuracy: 79.2%
Minibatch loss at step 1000: 97.997849
Minibatch accuracy: 78.9%
Validation accuracy: 81.8%
Minibatch loss at step 1500: 74.139778
Minibatch accuracy: 85.2%
Validation accuracy: 82.2%
Minibatch loss at step 2000: 57.005882
Minibatch accuracy: 89.8%
Validation accuracy: 82.6%
Minibatch loss at step 2500: 44.487976
Minibatch accuracy: 85.2%
Validation accuracy: 83.9%
Minibatch loss at step 3000: 34.578785
Minibatch accuracy: 85.2%
Validation accuracy: 84.2%
Test accuracy: 91.3%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [6]:
batch_size = 12

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [7]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 18.152536
Minibatch accuracy: 8.3%
Validation accuracy: 10.8%
Minibatch loss at step 500: 1.933836
Minibatch accuracy: 91.7%
Validation accuracy: 72.1%
Minibatch loss at step 1000: 0.268556
Minibatch accuracy: 91.7%
Validation accuracy: 69.4%
Minibatch loss at step 1500: 0.018284
Minibatch accuracy: 100.0%
Validation accuracy: 75.6%
Minibatch loss at step 2000: 4.813074
Minibatch accuracy: 75.0%
Validation accuracy: 73.0%
Minibatch loss at step 2500: 2.566367
Minibatch accuracy: 75.0%
Validation accuracy: 73.6%
Minibatch loss at step 3000: 2.651227
Minibatch accuracy: 83.3%
Validation accuracy: 75.3%
Test accuracy: 83.0%


In [8]:
batch_size = 12
hidden_layer_size = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weightsHidden = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer_size]))
  biasesHidden = tf.Variable(tf.zeros([hidden_layer_size]))
  
  logitsHidden = tf.matmul(tf_train_dataset, weightsHidden) + biasesHidden
  hiddenLayer = tf.nn.relu(logitsHidden)

  weights = tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits = tf.matmul(hiddenLayer, weights) + biases
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
 
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
    
  valid_hidden = tf.nn.relu(tf.matmul(tf_valid_dataset, weightsHidden) + biasesHidden)
  valid_logits = tf.matmul(valid_hidden, weights) + biases
  valid_prediction = tf.nn.softmax(valid_logits)
    
  test_hidden = tf.nn.relu(tf.matmul(tf_test_dataset, weightsHidden) + biasesHidden)
  test_logits = tf.matmul(test_hidden, weights) + biases
  test_prediction = tf.nn.softmax(test_logits)

In [9]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 343.291992
Minibatch accuracy: 0.0%
Validation accuracy: 12.5%
Minibatch loss at step 500: 403.188721
Minibatch accuracy: 75.0%
Validation accuracy: 57.8%
Minibatch loss at step 1000: 22.845827
Minibatch accuracy: 58.3%
Validation accuracy: 54.3%
Minibatch loss at step 1500: 44.549931
Minibatch accuracy: 75.0%
Validation accuracy: 56.5%
Minibatch loss at step 2000: 3119.778076
Minibatch accuracy: 66.7%
Validation accuracy: 57.9%
Minibatch loss at step 2500: 99.572533
Minibatch accuracy: 41.7%
Validation accuracy: 58.1%
Minibatch loss at step 3000: 239.537170
Minibatch accuracy: 66.7%
Validation accuracy: 56.3%
Test accuracy: 61.8%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [22]:
batch_size = 50
hidden_layer_size = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weightsHidden = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer_size]))
  biasesHidden = tf.Variable(tf.zeros([hidden_layer_size]))
  
  logitsHidden = tf.matmul(tf_train_dataset, weightsHidden) + biasesHidden
  hiddenLayer = tf.nn.relu(logitsHidden)

  # Add a 50% dropout during training only. Dropout also scales
  # activations such that no rescaling is needed at evaluation time.
  hidden = tf.nn.dropout(hiddenLayer, 0.5, seed=None)

  weights = tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))  
  
  # Training computation.
  logits = tf.matmul(hidden, weights) + biases
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # L2 regularization for the fully connected parameters.
  regularizers = (tf.nn.l2_loss(weightsHidden) + tf.nn.l2_loss(biasesHidden) +
                  tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases))
  # Add the regularization term to the loss.
  loss += 5e-4 * regularizers
 
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
    
  valid_hidden = tf.nn.relu(tf.matmul(tf_valid_dataset, weightsHidden) + biasesHidden)
  valid_logits = tf.matmul(valid_hidden, weights) + biases
  valid_prediction = tf.nn.softmax(valid_logits)
    
  test_hidden = tf.nn.relu(tf.matmul(tf_test_dataset, weightsHidden) + biasesHidden)
  test_logits = tf.matmul(test_hidden, weights) + biases
  test_prediction = tf.nn.softmax(test_logits)

In [23]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 569.167786
Minibatch accuracy: 8.0%
Validation accuracy: 23.2%
Minibatch loss at step 500: 252.663818
Minibatch accuracy: 68.0%
Validation accuracy: 72.8%
Minibatch loss at step 1000: 302.984375
Minibatch accuracy: 64.0%
Validation accuracy: 74.8%
Minibatch loss at step 1500: 236.571564
Minibatch accuracy: 68.0%
Validation accuracy: 74.4%
Minibatch loss at step 2000: 180.924744
Minibatch accuracy: 56.0%
Validation accuracy: 71.8%
Minibatch loss at step 2500: 112.501534
Minibatch accuracy: 72.0%
Validation accuracy: 70.3%
Minibatch loss at step 3000: 130.724213
Minibatch accuracy: 70.0%
Validation accuracy: 73.5%
Test accuracy: 80.8%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


# Use learning rate decay:

In [34]:
batch_size = 128
hidden_layer_size = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weightsHidden = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer_size]))
  biasesHidden = tf.Variable(tf.zeros([hidden_layer_size]))
  
  logitsHidden = tf.matmul(tf_train_dataset, weightsHidden) + biasesHidden
  hiddenLayer = tf.nn.relu(logitsHidden)



  # Add a 50% dropout during training only. Dropout also scales
  # activations such that no rescaling is needed at evaluation time.
  hidden = tf.nn.dropout(hiddenLayer, 0.5)

  weights = tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))  
  
  # Training computation.
  logits = tf.matmul(hidden, weights) + biases
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # L2 regularization for the fully connected parameters.
  regularizers = (tf.nn.l2_loss(weightsHidden) + tf.nn.l2_loss(biasesHidden) +
                  tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases))
  # Add the regularization term to the loss.
  loss += 5e-4 * regularizers
 
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 1, 0.9999)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  #optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
    
  valid_hidden = tf.nn.relu(tf.matmul(tf_valid_dataset, weightsHidden) + biasesHidden)
  valid_logits = tf.matmul(valid_hidden, weights) + biases
  valid_prediction = tf.nn.softmax(valid_logits)
    
  test_hidden = tf.nn.relu(tf.matmul(tf_test_dataset, weightsHidden) + biasesHidden)
  test_logits = tf.matmul(test_hidden, weights) + biases
  test_prediction = tf.nn.softmax(test_logits)

In [35]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 599.599609
Minibatch accuracy: 8.6%
Validation accuracy: 33.0%
Test accuracy: 35.6%

Minibatch loss at step 500: 142.883652
Minibatch accuracy: 75.0%
Validation accuracy: 79.9%
Minibatch loss at step 1000: 107.459633
Minibatch accuracy: 74.2%
Validation accuracy: 81.0%
Minibatch loss at step 1500: 81.212326
Minibatch accuracy: 79.7%
Validation accuracy: 79.3%
Minibatch loss at step 2000: 63.392540
Minibatch accuracy: 83.6%
Validation accuracy: 81.0%
Test accuracy: 88.5%

Minibatch loss at step 2500: 52.435539
Minibatch accuracy: 79.7%
Validation accuracy: 82.6%
Minibatch loss at step 3000: 42.954815
Minibatch accuracy: 78.9%
Validation accuracy: 83.0%
Minibatch loss at step 3500: 36.482975
Minibatch accuracy: 79.7%
Validation accuracy: 83.5%
Minibatch loss at step 4000: 30.742538
Minibatch accuracy: 81.2%
Validation accuracy: 83.5%
Test accuracy: 91.0%

Minibatch loss at step 4500: 26.143751
Minibatch accuracy: 82.0%
Validation accuracy: 84.3%
Mini

# Deeper Network along with learning rate decay:

# First try: 2 relu hidden layers with 1024 nodes. Use dropout 50%. Use learning rate decay. 

In [49]:
import math as math
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Hidden Layer 1, 1024 nodes
  weightsHidden1 = tf.Variable(tf.truncated_normal([image_size * image_size, 1024], stddev=math.sqrt(2.0/1024)))
  biasesHidden1 = tf.Variable(tf.zeros([1024]))
  logitsHidden1 = tf.matmul(tf_train_dataset, weightsHidden1) + biasesHidden1
  hiddenLayer1 = tf.nn.relu(logitsHidden1)

  # Hidden Layer 2, 1024 nodes
  weightsHidden2 = tf.Variable(tf.truncated_normal([1024, 1024], stddev=math.sqrt(2.0/1024)))
  biasesHidden2 = tf.Variable(tf.zeros([1024]))
  logitsHidden2 = tf.matmul(hiddenLayer1, weightsHidden2) + biasesHidden2
  hiddenLayer2 = tf.nn.relu(logitsHidden2)


  # Add a 50% dropout during training only. Dropout also scales
  # activations such that no rescaling is needed at evaluation time.
  hidden = tf.nn.dropout(hiddenLayer2, 0.5)

  # Training computation.
  weights = tf.Variable(tf.truncated_normal([1024, num_labels], stddev=math.sqrt(2.0/1024)))
  biases = tf.Variable(tf.zeros([num_labels]))  
  logits = tf.matmul(hidden, weights) + biases
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # L2 regularization for the fully connected parameters.
  regularizers = (tf.nn.l2_loss(weightsHidden1) + tf.nn.l2_loss(biasesHidden1) +
                  tf.nn.l2_loss(weightsHidden2) + tf.nn.l2_loss(biasesHidden2) +
                  tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases))
  # Add the regularization term to the loss.
  loss += 5e-4 * regularizers
 
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 1, 0.9999)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  #optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
    
  valid_hidden1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weightsHidden1) + biasesHidden1)
  valid_hidden2 = tf.nn.relu(tf.matmul(valid_hidden1, weightsHidden2) + biasesHidden2)
  valid_logits = tf.matmul(valid_hidden2, weights) + biases
  valid_prediction = tf.nn.softmax(valid_logits)
    
  test_hidden1 = tf.nn.relu(tf.matmul(tf_test_dataset, weightsHidden1) + biasesHidden1)
  test_hidden2 = tf.nn.relu(tf.matmul(test_hidden1, weightsHidden2) + biasesHidden2)
  test_logits = tf.matmul(test_hidden2, weights) + biases
  test_prediction = tf.nn.softmax(test_logits)

In [50]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Final Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.088232
Minibatch accuracy: 10.2%
Validation accuracy: 30.1%
Test accuracy: 32.8%

Minibatch loss at step 500: 0.969278
Minibatch accuracy: 88.3%
Validation accuracy: 85.5%
Minibatch loss at step 1000: 0.945249
Minibatch accuracy: 85.2%
Validation accuracy: 86.6%
Minibatch loss at step 1500: 0.669433
Minibatch accuracy: 92.2%
Validation accuracy: 87.8%
Minibatch loss at step 2000: 0.637509
Minibatch accuracy: 93.8%
Validation accuracy: 88.1%
Test accuracy: 94.5%

Minibatch loss at step 2500: 0.632518
Minibatch accuracy: 89.1%
Validation accuracy: 88.4%
Minibatch loss at step 3000: 0.631444
Minibatch accuracy: 87.5%
Validation accuracy: 89.0%
Minibatch loss at step 3500: 0.663117
Minibatch accuracy: 89.1%
Validation accuracy: 88.8%
Minibatch loss at step 4000: 0.502761
Minibatch accuracy: 93.8%
Validation accuracy: 89.2%
Test accuracy: 95.0%

Minibatch loss at step 4500: 0.488989
Minibatch accuracy: 90.6%
Validation accuracy: 89.0%
Minibatch loss a

# Second try:

In [53]:
import math as math
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Hidden Layer 1, 1024 nodes
  h1_size = 1024
  weightsHidden1 = tf.Variable(tf.truncated_normal([image_size * image_size, h1_size], stddev=math.sqrt(2.0/(image_size * image_size))))
  biasesHidden1 = tf.Variable(tf.zeros([h1_size]))
  logitsHidden1 = tf.matmul(tf_train_dataset, weightsHidden1) + biasesHidden1
  hiddenLayer1 = tf.nn.relu(logitsHidden1)

  # Hidden Layer 2, 1024 nodes
  h2_size = 1024
  weightsHidden2 = tf.Variable(tf.truncated_normal([h1_size, h2_size], stddev=math.sqrt(2.0/h1_size)))
  biasesHidden2 = tf.Variable(tf.zeros([h2_size]))
  logitsHidden2 = tf.matmul(hiddenLayer1, weightsHidden2) + biasesHidden2
  hiddenLayer2 = tf.nn.relu(logitsHidden2)

  # Hidden Layer 3, 305 nodes
  h3_size = 305
  weightsHidden3 = tf.Variable(tf.truncated_normal([h2_size, h3_size], stddev=math.sqrt(2.0/h2_size)))
  biasesHidden3 = tf.Variable(tf.zeros([h3_size]))
  logitsHidden3 = tf.matmul(hiddenLayer2, weightsHidden3) + biasesHidden3
  hiddenLayer3 = tf.nn.relu(logitsHidden3)
    
  # Hidden Layer 4, 75 nodes
  h4_size = 75
  weightsHidden4 = tf.Variable(tf.truncated_normal([h3_size, h4_size], stddev=math.sqrt(2.0/h3_size)))
  biasesHidden4 = tf.Variable(tf.zeros([h4_size]))
  logitsHidden4 = tf.matmul(hiddenLayer3, weightsHidden4) + biasesHidden4
  hiddenLayer4 = tf.nn.relu(logitsHidden4)


  # Add a 50% dropout during training only. Dropout also scales
  # activations such that no rescaling is needed at evaluation time.
  hidden = tf.nn.dropout(hiddenLayer4, 0.5)

  # Training computation.
  weights = tf.Variable(tf.truncated_normal([h4_size, num_labels], stddev=math.sqrt(2.0/h4_size)))
  biases = tf.Variable(tf.zeros([num_labels]))  
  logits = tf.matmul(hidden, weights) + biases
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # L2 regularization for the fully connected parameters.
  regularizers = (tf.nn.l2_loss(weightsHidden1) + tf.nn.l2_loss(biasesHidden1) +
                  tf.nn.l2_loss(weightsHidden2) + tf.nn.l2_loss(biasesHidden2) +
                  tf.nn.l2_loss(weightsHidden3) + tf.nn.l2_loss(biasesHidden3) +
                  tf.nn.l2_loss(weightsHidden4) + tf.nn.l2_loss(biasesHidden4) +
                  tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases))
  # Add the regularization term to the loss.
  loss += 1e-5 * regularizers
 
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 1, 0.9999)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  #optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
    
  valid_hidden1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weightsHidden1) + biasesHidden1)
  valid_hidden2 = tf.nn.relu(tf.matmul(valid_hidden1, weightsHidden2) + biasesHidden2)
  valid_hidden3 = tf.nn.relu(tf.matmul(valid_hidden2, weightsHidden3) + biasesHidden3)
  valid_hidden4 = tf.nn.relu(tf.matmul(valid_hidden3, weightsHidden4) + biasesHidden4)
  valid_logits = tf.matmul(valid_hidden4, weights) + biases
  valid_prediction = tf.nn.softmax(valid_logits)
    
  test_hidden1 = tf.nn.relu(tf.matmul(tf_test_dataset, weightsHidden1) + biasesHidden1)
  test_hidden2 = tf.nn.relu(tf.matmul(test_hidden1, weightsHidden2) + biasesHidden2)
  test_hidden3 = tf.nn.relu(tf.matmul(test_hidden2, weightsHidden3) + biasesHidden3)
  test_hidden4 = tf.nn.relu(tf.matmul(test_hidden3, weightsHidden4) + biasesHidden4)
  test_logits = tf.matmul(test_hidden4, weights) + biases
  test_prediction = tf.nn.softmax(test_logits)

In [54]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Final Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.442561
Minibatch accuracy: 11.7%
Validation accuracy: 20.4%
Test accuracy: 21.8%

Minibatch loss at step 500: 0.382377
Minibatch accuracy: 89.1%
Validation accuracy: 85.4%
Minibatch loss at step 1000: 0.515849
Minibatch accuracy: 85.2%
Validation accuracy: 86.4%
Minibatch loss at step 1500: 0.319223
Minibatch accuracy: 91.4%
Validation accuracy: 87.7%
Minibatch loss at step 2000: 0.277687
Minibatch accuracy: 93.0%
Validation accuracy: 88.5%
Test accuracy: 94.1%

Minibatch loss at step 2500: 0.356697
Minibatch accuracy: 89.8%
Validation accuracy: 88.5%
Minibatch loss at step 3000: 0.420701
Minibatch accuracy: 88.3%
Validation accuracy: 89.1%
Minibatch loss at step 3500: 0.370535
Minibatch accuracy: 90.6%
Validation accuracy: 89.0%
Minibatch loss at step 4000: 0.324853
Minibatch accuracy: 91.4%
Validation accuracy: 89.4%
Test accuracy: 95.0%

Minibatch loss at step 4500: 0.329836
Minibatch accuracy: 91.4%
Validation accuracy: 89.6%
Minibatch loss a

# Third try:

In [61]:
import math as math
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Hidden Layer 1, 1024 nodes
  h1_size = 1024
  weightsHidden1 = tf.Variable(tf.truncated_normal([image_size * image_size, h1_size], stddev=math.sqrt(2.0/(image_size * image_size))))
  biasesHidden1 = tf.Variable(tf.zeros([h1_size]))
  logitsHidden1 = tf.matmul(tf_train_dataset, weightsHidden1) + biasesHidden1
  hiddenLayer1 = tf.nn.relu(logitsHidden1)

  # Hidden Layer 2, 1024 nodes
  h2_size = 1024
  weightsHidden2 = tf.Variable(tf.truncated_normal([h1_size, h2_size], stddev=math.sqrt(2.0/h1_size)))
  biasesHidden2 = tf.Variable(tf.zeros([h2_size]))
  logitsHidden2 = tf.matmul(hiddenLayer1, weightsHidden2) + biasesHidden2
  hiddenLayer2 = tf.nn.relu(logitsHidden2)

  # Hidden Layer 3, 1024 nodes
  h3_size = 1024
  weightsHidden3 = tf.Variable(tf.truncated_normal([h2_size, h3_size], stddev=math.sqrt(2.0/h2_size)))
  biasesHidden3 = tf.Variable(tf.zeros([h3_size]))
  logitsHidden3 = tf.matmul(hiddenLayer2, weightsHidden3) + biasesHidden3
  hiddenLayer3 = tf.nn.relu(logitsHidden3)
    
  # Hidden Layer 4, 1024 nodes
  h4_size = 1024
  weightsHidden4 = tf.Variable(tf.truncated_normal([h3_size, h4_size], stddev=math.sqrt(2.0/h3_size)))
  biasesHidden4 = tf.Variable(tf.zeros([h4_size]))
  logitsHidden4 = tf.matmul(hiddenLayer3, weightsHidden4) + biasesHidden4
  hiddenLayer4 = tf.nn.relu(logitsHidden4)

  # Add a 50% dropout during training only. Dropout also scales
  # activations such that no rescaling is needed at evaluation time.
  hidden = tf.nn.dropout(hiddenLayer4, 0.5)

  # Training computation.
  weights = tf.Variable(tf.truncated_normal([h4_size, num_labels], stddev=math.sqrt(2.0/h4_size)))
  biases = tf.Variable(tf.zeros([num_labels]))  
  logits = tf.matmul(hidden, weights) + biases
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # L2 regularization for the fully connected parameters.
  regularizers = (tf.nn.l2_loss(weightsHidden1) + tf.nn.l2_loss(biasesHidden1) +
                  tf.nn.l2_loss(weightsHidden2) + tf.nn.l2_loss(biasesHidden2) +
                  tf.nn.l2_loss(weightsHidden3) + tf.nn.l2_loss(biasesHidden3) +
                  tf.nn.l2_loss(weightsHidden4) + tf.nn.l2_loss(biasesHidden4) +
                  tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases))
  # Add the regularization term to the loss.
  loss += 1e-5 * regularizers
 
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 1000, 0.9999)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  #optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
    
  valid_hidden1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weightsHidden1) + biasesHidden1)
  valid_hidden2 = tf.nn.relu(tf.matmul(valid_hidden1, weightsHidden2) + biasesHidden2)
  valid_hidden3 = tf.nn.relu(tf.matmul(valid_hidden2, weightsHidden3) + biasesHidden3)
  valid_hidden4 = tf.nn.relu(tf.matmul(valid_hidden3, weightsHidden4) + biasesHidden4)
  valid_logits = tf.matmul(valid_hidden4, weights) + biases
  valid_prediction = tf.nn.softmax(valid_logits)
    
  test_hidden1 = tf.nn.relu(tf.matmul(tf_test_dataset, weightsHidden1) + biasesHidden1)
  test_hidden2 = tf.nn.relu(tf.matmul(test_hidden1, weightsHidden2) + biasesHidden2)
  test_hidden3 = tf.nn.relu(tf.matmul(test_hidden2, weightsHidden3) + biasesHidden3)
  test_hidden4 = tf.nn.relu(tf.matmul(test_hidden3, weightsHidden4) + biasesHidden4)
  test_logits = tf.matmul(test_hidden4, weights) + biases
  test_prediction = tf.nn.softmax(test_logits)

In [63]:
num_steps = 95001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Final Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.446113
Minibatch accuracy: 10.2%
Validation accuracy: 25.2%
Test accuracy: 27.2%

Minibatch loss at step 500: 0.369400
Minibatch accuracy: 88.3%
Validation accuracy: 86.2%
Minibatch loss at step 1000: 0.536596
Minibatch accuracy: 83.6%
Validation accuracy: 86.6%
Minibatch loss at step 1500: 0.325853
Minibatch accuracy: 89.1%
Validation accuracy: 88.5%
Minibatch loss at step 2000: 0.305107
Minibatch accuracy: 93.0%
Validation accuracy: 88.3%
Test accuracy: 94.5%

Minibatch loss at step 2500: 0.347446
Minibatch accuracy: 90.6%
Validation accuracy: 88.7%
Minibatch loss at step 3000: 0.361364
Minibatch accuracy: 89.8%
Validation accuracy: 89.1%
Minibatch loss at step 3500: 0.368563
Minibatch accuracy: 88.3%
Validation accuracy: 89.5%
Minibatch loss at step 4000: 0.293228
Minibatch accuracy: 92.2%
Validation accuracy: 89.4%
Test accuracy: 95.2%

Minibatch loss at step 4500: 0.320054
Minibatch accuracy: 93.0%
Validation accuracy: 89.3%
Minibatch loss a